In [1]:
import pickle
from sklearn.metrics import confusion_matrix
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from csv import writer
import requests, bs4
import re
%matplotlib inline
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os
%load_ext Cython

In [2]:
team_abbrevs18 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
}

team_abbrevs17 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'San Antonio Stars': 'SAS'
}

team_abbrevs16 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'San Antonio Stars': 'SAS'
}

team_abbrevs15 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Tulsa Shock': 'TUL',
    'San Antonio Stars': 'SAS'
}

  

In [3]:
def date_formatter(date):

    if len(str(date.month)) != 2:
        month = '0'+ str(date.month)
    else:
        month = str(date.month)
        
    if len(str(date.day)) != 2:
        day = '0' + str(date.day)
    else:
        day = str(date.day)
        
    newdate = str(date.year) + month + day
    return newdate

In [4]:
def get_stats(team, year, team_abbrevs):
    
    url = f'https://www.basketball-reference.com/wnba/schedules/{team}/{year}.html'
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    reg_season = soup.find('table',{'id':'teams_games'})
    
    oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
    opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
    opp_abbrev = [team_abbrevs[opp] for opp in opps]
    
    datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
    dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date']
    
    souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
    soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})
    
    teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
    opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]
    
    teampoints = list(map(lambda x: int(x), teampoints))
    opppoints = list(map(lambda x: int(x), opppoints))
    
    gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
    locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']
    loc = list(map(lambda x: 1 if x =='' else 0, locs))
    
    result = [1 if (teampoints[i] - opppoints[i] > 0) else 0 for i in range(len(teampoints)) ]    
    
    links = reg_season.find_all(attrs = {'data-stat':'date_game'})
    urls = []
    for i in links:
        if i.find('a'):
            urls.append(i.find('a').attrs['href'])
    
    TSP = []
    TOV = []
    ORB = []
    FTR = []
    Poss = []
    OEff = []
    DEff = []
    
    for i, url in enumerate(urls):
        url = f'https://www.basketball-reference.com{url}'
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
        visitor = soup.find('div',{'id':'div_box-score-visitor'})
        home = soup.find('div',{'id':'div_box-score-home'})
        
        visitor = pd.read_html(str(visitor.find('table')), flavor='bs4')[0]
        home = pd.read_html(str(home.find('table')), flavor='bs4')[0]
        
        
        visitor = visitor.dropna().reset_index(drop=True)
        home = home.dropna().reset_index(drop = True)
        
        if loc[i] == 0:
            TSP.append(sum(visitor['PTS'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA']))*0.5)
            TOV.append(sum(visitor['TOV'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA'])+sum(visitor['TOV'])))
            ORB.append(sum(visitor['ORB'])/(sum(visitor['ORB'])+sum(home['TRB']-home['ORB'])))
            FTR.append(sum(visitor['FT'])/sum(visitor['FGA']))
            poss = 0.96 * (sum(visitor['FGA']) + sum(visitor['TOV']) + 0.44*sum(visitor['FTA']) - sum(visitor['ORB']))
            Poss.append(poss)
            OEff.append(100*(sum(visitor['PTS']))/poss)
            DEff.append(100*(sum(home['PTS']))/poss)
                       
        else:
            TSP.append(sum(home['PTS'])/(sum(home['FGA'])+0.44*sum(home['FTA']))*0.5)
            TOV.append(sum(home['TOV'])/(sum(home['FGA'])+0.44*sum(home['FTA'])+sum(home['TOV'])))
            ORB.append(sum(home['ORB'])/(sum(home['ORB'])+sum(visitor['TRB']-visitor['ORB'])))
            FTR.append(sum(home['FT'])/sum(home['FGA']))
            poss = 0.96 * (sum(home['FGA']) + sum(home['TOV']) + 0.44*sum(home['FTA']) - sum(home['ORB']))
            Poss.append(poss)
            OEff.append(100*(sum(home['PTS']))/poss)
            DEff.append(100*(sum(visitor['PTS']))/poss)
                       
    data = {"Team":[team] * 34, "Opp": opp_abbrev, "Game Num":list(range(1,35)), 'Date':dates, 
            'Team Score':teampoints, 'Opp Score':opppoints, 'Result': result,'Location':loc, 
            'TSP':TSP, 'TOV':TOV, 'ORB': ORB, 'FTR':FTR, 'Poss':Poss, 'OEff':OEff, 'DEff':DEff}
    return pd.DataFrame(data)

In [17]:
a = get_stats('ATL', '2015', team_abbrevs15)

In [5]:
df_2014 = pd.DataFrame()

for team in list(team_abbrevs15.values()):
    print(team)
    team_db = get_stats(team,'2014', team_abbrevs15)
    df_2014 = pd.concat([df_2014, team_db])

ATL
WAS
CON
CHI
NYL
IND
SEA
PHO
LAS
MIN
TUL
SAS


In [6]:
df_2014 = df_2014.reset_index(drop = True)
df_2014.to_csv('WNBA14.csv', index = False)

In [19]:
df_2015 = pd.DataFrame()

for team in list(team_abbrevs15.values()):
    print(team)
    team_db = get_stats(team,'2015', team_abbrevs15)
    df_2015 = pd.concat([df_2015, team_db])

ATL
WAS
CON
CHI
NYL
IND
SEA
PHO
LAS
MIN
TUL
SAS


In [20]:
df_2015 = df_2015.reset_index(drop = True)

In [21]:
df_2015.to_csv('WNBA15.csv', index = False)

In [24]:
df_2016 = pd.DataFrame()

for team in list(team_abbrevs16.values()):
    print(team)
    team_db = get_stats(team,'2016', team_abbrevs16)
    df_2016 = pd.concat([df_2016, team_db])
    
df_2016 = df_2016.reset_index(drop = True)
df_2016.to_csv('WNBA16.csv', index = False)

ATL
WAS
CON
CHI
NYL
IND
SEA
PHO
LAS
MIN
DAL
SAS


In [25]:
df_2017 = pd.DataFrame()

for team in list(team_abbrevs17.values()):
    print(team)
    team_db = get_stats(team,'2017', team_abbrevs17)
    df_2017 = pd.concat([df_2017, team_db])
    
df_2017 = df_2017.reset_index(drop = True)
df_2017.to_csv('WNBA17.csv', index = False)

ATL
WAS
CON
CHI
NYL
IND
SEA
PHO
LAS
MIN
DAL
SAS


In [48]:
team_abbrevs18 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
}

df_2018 = pd.DataFrame()

for team in ['ATL', 'CON', 'CHI', 'NYL', 'IND', 'SEA', 'PHO', 'MIN', 'LAS', 'DAL']:
    print(team)
    team_db = get_stats(team,'2018', team_abbrevs18)
    df_2018 = pd.concat([df_2018, team_db])

df_2018 = df_2018.reset_index(drop = True)
df_2018 = pd.concat([df_2018, was, las])
df_2018 = df_2018.reset_index(drop = True)
df_2018.to_csv('WNBA18.csv', index = False)

ATL
CON
CHI
NYL
IND
SEA
PHO
MIN
LAS
DAL


### Below is for troubleshooting, grabbing Washington and LV manually

In [28]:
was = get_stats('WAS', '2018', team_abbrevs18)

ValueError: arrays must all be same length

In [29]:
url = 'https://www.basketball-reference.com/wnba/schedules/WAS/2018.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')

In [30]:
reg_season = soup.find('table',{'id':'teams_games'})

In [33]:
oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
opp_abbrev = [team_abbrevs18[opp] for opp in opps]
print(len(opp_abbrev))

34


In [34]:
datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date']
print(len(dates))

34


In [38]:
souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})
teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
teampoints.insert(26, np.nan)
print(len(teampoints))
opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]
opppoints.insert(26, np.nan)
print(len(opppoints))

gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']
loc = list(map(lambda x: 1 if x =='' else 0, locs))
print(len(loc))

34
34
34


In [13]:
test = {'Team': ['ATL']*34, 'Team Pts':teampoints, 'Opp Pts':opppoints, 'Location': loc, 'Date':dates}

In [14]:
pd.DataFrame(test)

,Date,Location,Opp Pts,Team,Team Pts
0,"Sun, May 20, 2018",0,101,ATL,78
1,"Wed, May 23, 2018",0,63,ATL,81
2,"Sat, May 26, 2018",1,78,ATL,70
3,"Tue, May 29, 2018",1,74,ATL,76
4,"Sun, Jun 3, 2018",1,78,ATL,71
5,"Tue, Jun 5, 2018",1,77,ATL,82
6,"Fri, Jun 8, 2018",0,83,ATL,87
7,"Sun, Jun 10, 2018",0,64,ATL,67
8,"Tue, Jun 12, 2018",0,72,ATL,64
9,"Thu, Jun 14, 2018",1,67,ATL,72


In [15]:
test = reg_season.find_all(attrs = {'data-stat':'date_game'})

In [16]:
urls = []
for i in test:
    if i.find('a'):
        urls.append(i.find('a').attrs['href'])

In [17]:
dct = {'URL': urls, 'Team': ['ATL']*34, 'Team Pts':teampoints, 'Opp Pts':opppoints, 'Location': loc, 'Date':dates}

In [18]:
pd.DataFrame(dct)

,Date,Location,Opp Pts,Team,Team Pts,URL
0,"Sun, May 20, 2018",0,101,ATL,78,/wnba/boxscores/201805200DAL.html
1,"Wed, May 23, 2018",0,63,ATL,81,/wnba/boxscores/201805230CHI.html
2,"Sat, May 26, 2018",1,78,ATL,70,/wnba/boxscores/201805260ATL.html
3,"Tue, May 29, 2018",1,74,ATL,76,/wnba/boxscores/201805290ATL.html
4,"Sun, Jun 3, 2018",1,78,ATL,71,/wnba/boxscores/201806030ATL.html
5,"Tue, Jun 5, 2018",1,77,ATL,82,/wnba/boxscores/201806050ATL.html
6,"Fri, Jun 8, 2018",0,83,ATL,87,/wnba/boxscores/201806080LVA.html
7,"Sun, Jun 10, 2018",0,64,ATL,67,/wnba/boxscores/201806100SEA.html
8,"Tue, Jun 12, 2018",0,72,ATL,64,/wnba/boxscores/201806120LAS.html
9,"Thu, Jun 14, 2018",1,67,ATL,72,/wnba/boxscores/201806140ATL.html


In [33]:
url = 'https://www.basketball-reference.com/wnba/boxscores/201805200DAL.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
visitor = soup.find('div',{'id':'div_box-score-visitor'})
home = soup.find('div',{'id':'div_box-score-home'})

In [34]:
tdf = pd.read_html(str(visitor.find('table')), flavor='bs4')[0]
home = pd.read_html(str(home.find('table')), flavor='bs4')[0]

In [35]:
tdf = tdf.dropna()
home = home.dropna()

In [41]:
tdf = tdf.reset_index(drop=True)
home = home.reset_index(drop = True)

In [29]:
TSP = sum(tdf['PTS'])/(sum(tdf['FGA'])+0.44*sum(tdf['FTA']))*0.5

In [31]:
TOV = sum(tdf['TOV'])/(sum(tdf['FGA'])+0.44*sum(tdf['FTA'])+sum(tdf['TOV']))

In [44]:
ORB = sum(tdf['ORB'])/(sum(tdf['ORB'])+sum(home['TRB']-home['ORB']))

In [47]:
FTR = sum(tdf['FT'])/sum(tdf['FGA'])

In [48]:
FTR

0.2876712328767123

In [52]:
poss = 0.96 * (sum(tdf['FGA']) + sum(tdf['TOV']) + 0.44*sum(tdf['FTA']) - sum(tdf['ORB']))

In [54]:
OEff = 100*(sum(tdf['PTS']))/poss

In [56]:
DEff = 100*(sum(home['PTS']))/poss

## Manually getting Washington and Las Vegas 2018 bc they had a forfeited game

In [44]:
team = 'WAS'
url = f'https://www.basketball-reference.com/wnba/schedules/WAS/2018.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
reg_season = soup.find('table',{'id':'teams_games'})
    
oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
opp_abbrev = [team_abbrevs18[opp] for opp in opps]

datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date']

souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})

teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]
teampoints = list(map(lambda x: int(x), teampoints))
opppoints = list(map(lambda x: int(x), opppoints))

teampoints.insert(26, np.nan)
opppoints.insert(26, np.nan)

gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']
loc = list(map(lambda x: 1 if x =='' else 0, locs))

result = [1 if (teampoints[i] - opppoints[i] > 0) else 0 for i in range(len(teampoints))]

print(len(result))
print(len(loc))

34
34


In [45]:
links = reg_season.find_all(attrs = {'data-stat':'date_game'})
urls = []
for i in links:
    if i.find('a'):
        urls.append(i.find('a').attrs['href'])
    
TSP = []
TOV = []
ORB = []
FTR = []
Poss = []
OEff = []
DEff = []
    
for i, url in enumerate(urls):
    url = f'https://www.basketball-reference.com{url}'
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    visitor = soup.find('div',{'id':'div_box-score-visitor'})
    home = soup.find('div',{'id':'div_box-score-home'})
        
    visitor = pd.read_html(str(visitor.find('table')), flavor='bs4')[0]
    home = pd.read_html(str(home.find('table')), flavor='bs4')[0]

    visitor = visitor.dropna().reset_index(drop=True)
    home = home.dropna().reset_index(drop = True)
        
    if loc[i] == 0:
        TSP.append(sum(visitor['PTS'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA']))*0.5)
        TOV.append(sum(visitor['TOV'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA'])+sum(visitor['TOV'])))
        ORB.append(sum(visitor['ORB'])/(sum(visitor['ORB'])+sum(home['TRB']-home['ORB'])))
        FTR.append(sum(visitor['FT'])/sum(visitor['FGA']))
        poss = 0.96 * (sum(visitor['FGA']) + sum(visitor['TOV']) + 0.44*sum(visitor['FTA']) - sum(visitor['ORB']))
        Poss.append(poss)
        OEff.append(100*(sum(visitor['PTS']))/poss)
        DEff.append(100*(sum(home['PTS']))/poss)
                       
    else:
        TSP.append(sum(home['PTS'])/(sum(home['FGA'])+0.44*sum(home['FTA']))*0.5)
        TOV.append(sum(home['TOV'])/(sum(home['FGA'])+0.44*sum(home['FTA'])+sum(home['TOV'])))
        ORB.append(sum(home['ORB'])/(sum(home['ORB'])+sum(visitor['TRB']-visitor['ORB'])))
        FTR.append(sum(home['FT'])/sum(home['FGA']))
        poss = 0.96 * (sum(home['FGA']) + sum(home['TOV']) + 0.44*sum(home['FTA']) - sum(home['ORB']))
        Poss.append(poss)
        OEff.append(100*(sum(home['PTS']))/poss)
        DEff.append(100*(sum(visitor['PTS']))/poss)

TSP.insert(26, np.nan)
TOV.insert(26, np.nan)
ORB.insert(26, np.nan)
FTR.insert(26, np.nan)
Poss.insert(26, np.nan)
OEff.insert(26, np.nan)
DEff.insert(26, np.nan)

data = {"Team":[team] * 34, "Opp": opp_abbrev, "Game Num":list(range(1,35)), 'Date':dates, 
        'Team Score':teampoints, 'Opp Score':opppoints, 'Result': result,'Location':loc, 
        'TSP':TSP, 'TOV':TOV, 'ORB': ORB, 'FTR':FTR, 'Poss':Poss, 'OEff':OEff, 'DEff':DEff}

was = pd.DataFrame(data)

In [49]:
team = 'LVA'
url = f'https://www.basketball-reference.com/wnba/schedules/LVA/2018.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
reg_season = soup.find('table',{'id':'teams_games'})
    
oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
opp_abbrev = [team_abbrevs18[opp] for opp in opps]

datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date']

souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})

teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]
teampoints = list(map(lambda x: int(x), teampoints))
opppoints = list(map(lambda x: int(x), opppoints))

teampoints.insert(26, np.nan)
opppoints.insert(26, np.nan)

gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']
loc = list(map(lambda x: 1 if x =='' else 0, locs))

result = [1 if (teampoints[i] - opppoints[i] > 0) else 0 for i in range(len(teampoints))]

links = reg_season.find_all(attrs = {'data-stat':'date_game'})
urls = []
for i in links:
    if i.find('a'):
        urls.append(i.find('a').attrs['href'])
    
TSP = []
TOV = []
ORB = []
FTR = []
Poss = []
OEff = []
DEff = []
    
for i, url in enumerate(urls):
    url = f'https://www.basketball-reference.com{url}'
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    visitor = soup.find('div',{'id':'div_box-score-visitor'})
    home = soup.find('div',{'id':'div_box-score-home'})
        
    visitor = pd.read_html(str(visitor.find('table')), flavor='bs4')[0]
    home = pd.read_html(str(home.find('table')), flavor='bs4')[0]

    visitor = visitor.dropna().reset_index(drop=True)
    home = home.dropna().reset_index(drop = True)
        
    if loc[i] == 0:
        TSP.append(sum(visitor['PTS'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA']))*0.5)
        TOV.append(sum(visitor['TOV'])/(sum(visitor['FGA'])+0.44*sum(visitor['FTA'])+sum(visitor['TOV'])))
        ORB.append(sum(visitor['ORB'])/(sum(visitor['ORB'])+sum(home['TRB']-home['ORB'])))
        FTR.append(sum(visitor['FT'])/sum(visitor['FGA']))
        poss = 0.96 * (sum(visitor['FGA']) + sum(visitor['TOV']) + 0.44*sum(visitor['FTA']) - sum(visitor['ORB']))
        Poss.append(poss)
        OEff.append(100*(sum(visitor['PTS']))/poss)
        DEff.append(100*(sum(home['PTS']))/poss)
                       
    else:
        TSP.append(sum(home['PTS'])/(sum(home['FGA'])+0.44*sum(home['FTA']))*0.5)
        TOV.append(sum(home['TOV'])/(sum(home['FGA'])+0.44*sum(home['FTA'])+sum(home['TOV'])))
        ORB.append(sum(home['ORB'])/(sum(home['ORB'])+sum(visitor['TRB']-visitor['ORB'])))
        FTR.append(sum(home['FT'])/sum(home['FGA']))
        poss = 0.96 * (sum(home['FGA']) + sum(home['TOV']) + 0.44*sum(home['FTA']) - sum(home['ORB']))
        Poss.append(poss)
        OEff.append(100*(sum(home['PTS']))/poss)
        DEff.append(100*(sum(visitor['PTS']))/poss)

TSP.insert(26, np.nan)
TOV.insert(26, np.nan)
ORB.insert(26, np.nan)
FTR.insert(26, np.nan)
Poss.insert(26, np.nan)
OEff.insert(26, np.nan)
DEff.insert(26, np.nan)

data = {"Team":[team] * 34, "Opp": opp_abbrev, "Game Num":list(range(1,35)), 'Date':dates, 
        'Team Score':teampoints, 'Opp Score':opppoints, 'Result': result,'Location':loc, 
        'TSP':TSP, 'TOV':TOV, 'ORB': ORB, 'FTR':FTR, 'Poss':Poss, 'OEff':OEff, 'DEff':DEff}

las = pd.DataFrame(data)

In [51]:
db = pd.read_csv('WNBA18.csv')

In [52]:
db = pd.concat([db, las])

In [55]:
db = db.reset_index(drop = True)

In [56]:
db.to_csv('WNBA18.csv', index = False)